In [1]:
import autogen

In [2]:
llm_config = {"model": "gpt-4o-mini"}

In [3]:
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

False

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)

In [5]:
import os
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.7,
    openai_api_key=os.getenv("OPENAI_API_KEY")
)

In [6]:
from langchain.chains import LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)
from langchain.memory import ConversationBufferMemory

class Agent:
    def __init__(self, name, system_prompt, llm):
        self.name = name
        self.llm = llm
        self.system_prompt = system_prompt
        # Create a chat prompt that includes 'conversation_history'
        self.chat_prompt = ChatPromptTemplate.from_messages([
            SystemMessagePromptTemplate.from_template(self.system_prompt),
            # Include the conversation history and the last message in the prompt
            HumanMessagePromptTemplate.from_template("{conversation_history}\n{input}")
        ])
        self.chain = LLMChain(
            llm=self.llm,
            prompt=self.chat_prompt,
            verbose=False
        )

    def respond(self, input_text, conversation_history):
        response = self.chain.predict(
            input=input_text,
            conversation_history=conversation_history
        )
        return response

In [7]:
# Define each agent with system-specific prompts
customer_proxy_agent = Agent(
    "Customer_Proxy", 
    "Communicate customer requirements to the Project Manager regarding the DevSecOps pipeline, including CI/CD, security integration, IaC, and compliance.", 
    llm=llm
)

project_manager_agent = Agent(
    "Project_Manager", 
    "Coordinate project phases, relay tasks to each engineer, and ensure project milestones are met for the DevSecOps pipeline.", 
    llm=llm
)

requirement_engineer_agent = Agent(
    "Requirement_Engineer", 
    "Gather and document detailed requirements for the DevSecOps pipeline, including use cases for CI/CD automation, security integration, infrastructure-as-code (IaC), and compliance tracking. Calculate effort based on documented requirements.", 
    llm=llm
)

system_engineer_agent = Agent(
    "System_Engineer", 
    "Create a detailed design document for the DevSecOps pipeline, covering architecture for CI/CD, security integration, and IaC. Calculate effort based on the design document.", 
    llm=llm
)

software_engineer_agent = Agent(
    "Software_Engineer", 
    "Develop the software based on the design document, focusing on automation for CI/CD, security integration, and infrastructure provisioning. Calculate effort based on source lines of code (SLOC).", 
    llm=llm
)

test_engineer_agent = Agent(
    "Test_Engineer", 
    "Create a test plan with detailed test cases for the DevSecOps pipeline and execute tests based on the software, design, and requirements documents. Calculate effort based on executed test cases.", 
    llm=llm
)

documentation_engineer_agent = Agent(
    "Documentation_Engineer", 
    "Prepare comprehensive user documentation and training materials for the DevSecOps pipeline, covering all features and processes. Calculate effort based on documentation pages created.", 
    llm=llm
)


C:\Users\SEJAL SRIVASTAV\AppData\Local\Temp\ipykernel_17304\3812591458.py:20: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  self.chain = LLMChain(


In [8]:
nested_chat_queue = [
    {
        "sender": project_manager_agent, 
        "recipient": requirement_engineer_agent, 
        "message": (
            "I have received the customer requirements from the Customer Proxy. Please document detailed use-cases and requirements for the DevSecOps pipeline, "
            "covering CI/CD automation, security integration, infrastructure-as-code (IaC), and compliance tracking. "
            "Ensure clarity and completeness in defining each requirement, and provide estimates for the effort required."
        )
    },
    {
        "sender": project_manager_agent, 
        "recipient": system_engineer_agent, 
        "message": (
            "The Requirement Engineer has completed documenting the requirements. Proceed with creating the system design document for the DevSecOps pipeline, "
            "covering architecture for CI/CD automation, security integration, and IaC. Include effort estimates based on the design document."
        )
    },
    {
        "sender": project_manager_agent, 
        "recipient": software_engineer_agent, 
        "message": (
            "The System Engineer has completed the design document. Proceed with developing the software for the DevSecOps pipeline based on the design, "
            "focusing on automation for CI/CD, security integration, and IaC. Provide detailed estimates for the development phase effort."
        )
    },
    {
        "sender": project_manager_agent, 
        "recipient": test_engineer_agent, 
        "message": (
            "The software has been developed. Please create a detailed test plan and execute test cases for the DevSecOps pipeline, based on the software, design, and requirements documents. "
            "Provide effort estimates for the testing phase."
        )
    },
    {
        "sender": project_manager_agent, 
        "recipient": documentation_engineer_agent, 
        "message": (
            "Testing is complete. Please proceed with creating user documentation and training materials for the DevSecOps pipeline, covering all features and processes. "
            "Provide effort estimates for the documentation phase."
        )
    },
]


In [9]:
# Initialize conversation history and storage for each agent's output
conversation_history = ""
engineer_outputs = {}

# Function to handle nested chat flow with user control
def initiate_nested_chats():
    # Customer Proxy initiates conversation with Project Manager
    initial_message = (
        "I want to create a fully automated DevSecOps pipeline for our organization. "
        "The pipeline should handle continuous integration, delivery, and deployment, integrate security tools like SAST/DAST, "
        "automate infrastructure provisioning using Infrastructure-as-Code (IaC), and ensure compliance and audit logging throughout the process."
    )
    print(f"Customer_Proxy:\n{initial_message}\n")
    conversation_history = f"Customer_Proxy: {initial_message}\n"
    
    # Start conversation flow through each agent in nested_chat_queue
    accumulated_work = ""
    
    for chat in nested_chat_queue:
        recipient = chat['recipient']
        message = chat['message']
        
        # Include accumulated work from previous agents
        if accumulated_work:
            message += f"\n\nHere is the combined work from previous phases:\n{accumulated_work}"
        
        print(f"Project_Manager to {recipient.name}: {message}")
        
        # Initiate chat and capture response
        response = project_manager_agent.respond(message, conversation_history=conversation_history)
        print(f"{recipient.name}:\n{response}\n")
        
        # Update conversation history and store response
        conversation_history += f"Project_Manager: {message}\n{recipient.name}: {response}\n"
        engineer_outputs[recipient.name] = response
        
        # Accumulate response for use in the next phase
        accumulated_work += f"\n\nWork from {recipient.name}:\n{response}"
        
        # Ask the user if they want to continue or exit
        user_input = input("Press 'Enter' to continue the conversation, or type 'exit' to end it: ")
        if user_input.lower() == 'exit':
            print("Conversation ended by user.")
            return  # Exit the function if user chooses to end

    # Compile final project plan if user chose to complete all phases
    final_project_plan = compile_final_project_plan(engineer_outputs)

    # Project Manager sends final message to Customer Proxy
    final_message_to_customer = f"Final Project Plan:\n\n{final_project_plan}"
    print(f"Project_Manager to Customer_Proxy:\n{final_message_to_customer}")
    conversation_history += f"Project_Manager: {final_message_to_customer}\n"


In [10]:
# Initialize conversation history and storage for each agent's output
conversation_history = ""
engineer_outputs = {}

# Function to extract engineer's work from response
def extract_engineer_work(response):
    return response if response else "No output available."

# Function to handle nested chat flow with user control
def initiate_nested_chats():
    # Customer Proxy initiates conversation with Project Manager
    initial_message = (
        "I want to create a fully automated DevSecOps pipeline for our organization. "
        "The pipeline should handle continuous integration, delivery, and deployment, integrate security tools like SAST/DAST, "
        "automate infrastructure provisioning using Infrastructure-as-Code (IaC), and ensure compliance and audit logging throughout the process."
    )
    print(f"Customer_Proxy:\n{initial_message}\n")
    conversation_history = f"Customer_Proxy: {initial_message}\n"
    
    # Start conversation flow through each agent in nested_chat_queue
    accumulated_work = ""
    
    for chat in nested_chat_queue:
        recipient = chat['recipient']
        message = chat['message']
        
        # Include accumulated work from previous agents
        if accumulated_work:
            message += f"\n\nHere is the combined work from previous phases:\n{accumulated_work}"
        
        print(f"Project_Manager to {recipient.name}: {message}")
        
        # Initiate chat and capture response
        response = project_manager_agent.respond(message, conversation_history=conversation_history)
        print(f"{recipient.name}:\n{response}\n")
        
        # Update conversation history and store response
        conversation_history += f"Project_Manager: {message}\n{recipient.name}: {response}\n"
        engineer_outputs[recipient.name] = response
        
        # Accumulate response for use in the next phase
        accumulated_work += f"\n\nWork from {recipient.name}:\n{response}"
        
        # Ask the user if they want to continue or exit
        user_input = input("Press 'Enter' to continue the conversation, or type 'exit' to end it: ")
        if user_input.lower() == 'exit':
            print("Conversation ended by user.")
            return  # Exit the function if user chooses to end

    # Compile final project plan if user chose to complete all phases
    final_project_plan = compile_final_project_plan(engineer_outputs)

    # Project Manager sends final message to Customer Proxy
    final_message_to_customer = f"Final Project Plan:\n\n{final_project_plan}"
    print(f"Project_Manager to Customer_Proxy:\n{final_message_to_customer}")
    conversation_history += f"Project_Manager: {final_message_to_customer}\n"

# Function to compile the final project plan based on engineer outputs
def compile_final_project_plan(engineer_outputs):
    final_plan = "### Final Project Plan\n\nThe project has been completed successfully. Below are the contributions from each phase:\n"
    for phase, work in engineer_outputs.items():
        final_plan += f"\n#### {phase} Phase:\n{work}\n"
    final_plan += "\nThe project is ready for deployment."
    return final_plan

# Start the nested conversation
initiate_nested_chats()


Customer_Proxy:
I want to create a fully automated DevSecOps pipeline for our organization. The pipeline should handle continuous integration, delivery, and deployment, integrate security tools like SAST/DAST, automate infrastructure provisioning using Infrastructure-as-Code (IaC), and ensure compliance and audit logging throughout the process.

Project_Manager to Requirement_Engineer: I have received the customer requirements from the Customer Proxy. Please document detailed use-cases and requirements for the DevSecOps pipeline, covering CI/CD automation, security integration, infrastructure-as-code (IaC), and compliance tracking. Ensure clarity and completeness in defining each requirement, and provide estimates for the effort required.
Requirement_Engineer:
# DevSecOps Pipeline Requirements Document

## Overview
The goal of this document is to outline detailed use cases and requirements for a fully automated DevSecOps pipeline that encompasses continuous integration (CI), continuous

Press 'Enter' to continue the conversation, or type 'exit' to end it:  exit


Conversation ended by user.


In [ ]:
from langchain.chains import LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)
from langchain.memory import ConversationBufferMemory

class Agent:
    def __init__(self, name, system_message, llm):
        self.name = name
        self.system_message = system_message
        self.llm = llm
        # Create a chat prompt that includes 'conversation_history'
        self.chat_prompt = ChatPromptTemplate.from_messages([
            SystemMessagePromptTemplate.from_template(self.system_message),
            HumanMessagePromptTemplate.from_template("{conversation_history}\n{input}")
        ])
        self.chain = LLMChain(
            llm=self.llm,
            prompt=self.chat_prompt,
            verbose=False
        )

    def respond(self, task_message, conversation_history):
        # Request the model to assume realistic effort estimates based on role
        response = self.chain.predict(
            input=task_message,
            conversation_history=conversation_history
        )
        return response

# Define agents with prompts that allow for inferred effort estimation
product_owner_agent = Agent(
    name="Product_Owner", 
    system_message="""Define and prioritize the requirements for the DevSecOps pipeline, focusing on CI/CD, security, IaC, and compliance. Provide estimates based on standard project phases.""",
    llm=llm
)

scrum_master_agent = Agent(
    name="Scrum_Master", 
    system_message="""Facilitate sprint planning, daily standups, sprint reviews, and retrospectives. Ensure the Development Team stays focused on sprint goals for the DevSecOps pipeline.""",
    llm=llm
)

# Define Development Team members including new roles
development_team_agents = {
    "Requirement_Engineer": Agent(
        name="Requirement_Engineer", 
        system_message="""Define and document all requirements for the DevSecOps pipeline based on project goals.
                          Estimate time and effort using typical practices for similar tasks in DevSecOps projects.""",
        llm=llm
    ),
    "System_Engineer": Agent(
        name="System_Engineer", 
        system_message="""Create the design document, covering architecture for CI/CD, security, and IaC, based on standard practices.
                          Estimate time required based on general effort for design tasks in DevSecOps projects.""",
        llm=llm
    ),
    "Software_Engineer": Agent(
        name="Software_Engineer", 
        system_message="""Develop the software for the DevSecOps pipeline based on design specifications. Estimate required time based on similar automation projects in DevSecOps.""",
        llm=llm
    ),
    "Test_Engineer": Agent(
        name="Test_Engineer", 
        system_message="""Develop a comprehensive test plan and execute tests for all requirements in the DevSecOps pipeline.
                          Provide an estimated time frame based on typical testing durations for similar projects.""",
        llm=llm
    ),
    "Documentation_Engineer": Agent(
        name="Documentation_Engineer", 
        system_message="""Create user documentation and training materials for the software project, based on standard DevSecOps project timelines.""",
        llm=llm
    ),
    "Release_Manager": Agent(
        name="Release_Manager", 
        system_message="""Coordinate deployment and release activities, ensuring that all components of the DevSecOps pipeline are ready for production deployment.
                          Estimate time based on industry standards for release management.""",
        llm=llm
    ),
    "Architect": Agent(
        name="Architect", 
        system_message="""Develop high-level system architecture for the DevSecOps pipeline, ensuring scalability, performance, and security.
                          Estimate time based on standard architectural practices for similar projects.""",
        llm=llm
    ),
    "Business_Analyst": Agent(
        name="Business_Analyst", 
        system_message="""Collaborate with stakeholders to gather and document business requirements for the DevSecOps pipeline, ensuring alignment with organizational goals.
                          Estimate time based on typical business analysis tasks in similar projects.""",
        llm=llm
    )
}

# Function to handle sprint flow
def run_sprint(sprint_number):
    global conversation_history
    print(f"\n=== Sprint {sprint_number} ===\n")
    
    # Sprint Planning
    sprint_goal = product_owner_agent.respond(
        "Define the sprint goal and prioritize the backlog for the DevSecOps pipeline.",
        conversation_history
    )
    print(f"Product_Owner:\n{sprint_goal}\n")
    conversation_history += f"Product_Owner: {sprint_goal}\n"
    
    # Assign initial tasks to each Development Team agent with inferred effort
    for role, agent in development_team_agents.items():
        task_message = f"Sprint task for {role}: Work on user stories related to {role} responsibilities. Provide an estimated time frame for your tasks."
        response = agent.respond(task_message, conversation_history)
        print(f"{role}:\n{response}\n")
        conversation_history += f"{role}: {response}\n"

    # Daily Standups
 # Single Daily Standup
    print(f"\n--- Daily Standup ---")
    for role, agent in development_team_agents.items():
        standup_message = f"Daily Standup for {role}: Share progress, blockers, and next steps."
        standup_response = agent.respond(standup_message, conversation_history)
        print(f"{role}:\n{standup_response}\n")
        conversation_history += f"{role}: {standup_response}\n"

    
    # Sprint Review
    review_message = "Sprint Review: Present the completed work and user stories."
    review_response = scrum_master_agent.respond(review_message, conversation_history)
    print(f"Scrum_Master:\n{review_response}\n")
    conversation_history += f"Scrum_Master: {review_response}\n"
    
    # Sprint Retrospective
    retrospective_message = "Sprint Retrospective: Discuss what went well, areas for improvement, and blockers."
    retrospective_response = scrum_master_agent.respond(retrospective_message, conversation_history)
    print(f"Scrum_Master:\n{retrospective_response}\n")
    conversation_history += f"Scrum_Master: {retrospective_response}\n"

# Execute a single sprint
number_of_sprints = 1
run_sprint(number_of_sprints)



=== Sprint 1 ===

Product_Owner:
### Sprint Goal
The primary goal for this sprint is to enhance the DevSecOps pipeline by successfully implementing key features that integrate Continuous Integration/Continuous Deployment (CI/CD), security practices, Infrastructure as Code (IaC), and compliance checks. The aim is to ensure a secure, efficient, and compliant software delivery process while providing comprehensive documentation and training materials for the team.

### Prioritized Backlog for the DevSecOps Pipeline

1. **User Story 1: CI/CD Implementation**
   - **As a** DevOps Engineer, **I want** to set up a CI/CD pipeline using Jenkins/GitLab CI to automate build, test, and deployment processes.
   - **Estimated Effort**: 10 days

2. **User Story 2: Security Integration**
   - **As a** Security Engineer, **I want** to integrate security tools (e.g., SAST, DAST) into the CI/CD pipeline to identify vulnerabilities early in the development process.
   - **Estimated Effort**: 7 days

3. *